In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import random
from pprint import pp
import re
import tensorflow as tf

In [2]:
urls = ['https://allthings.umphreys.com/setlists/' + str(year) for year in range(1998,2023)]
urls

['https://allthings.umphreys.com/setlists/1998',
 'https://allthings.umphreys.com/setlists/1999',
 'https://allthings.umphreys.com/setlists/2000',
 'https://allthings.umphreys.com/setlists/2001',
 'https://allthings.umphreys.com/setlists/2002',
 'https://allthings.umphreys.com/setlists/2003',
 'https://allthings.umphreys.com/setlists/2004',
 'https://allthings.umphreys.com/setlists/2005',
 'https://allthings.umphreys.com/setlists/2006',
 'https://allthings.umphreys.com/setlists/2007',
 'https://allthings.umphreys.com/setlists/2008',
 'https://allthings.umphreys.com/setlists/2009',
 'https://allthings.umphreys.com/setlists/2010',
 'https://allthings.umphreys.com/setlists/2011',
 'https://allthings.umphreys.com/setlists/2012',
 'https://allthings.umphreys.com/setlists/2013',
 'https://allthings.umphreys.com/setlists/2014',
 'https://allthings.umphreys.com/setlists/2015',
 'https://allthings.umphreys.com/setlists/2016',
 'https://allthings.umphreys.com/setlists/2017',
 'https://allthings.

In [3]:
setlists = []

for url in urls:
    webpage = requests.get(url)
    soup = BeautifulSoup(webpage.content,'html.parser')
    matches = soup.find_all(class_='setlist setlist-container')
    for match in matches:
        setlists.append(match.get_text())

In [4]:
setlists_numpy = np.array(setlists)

In [5]:
len(setlists)

2603

In [6]:
import sys
sys.getsizeof(setlists)

23128

In [7]:
from keras.preprocessing.text import Tokenizer

In [8]:
setlists_pretoken = [re.sub(r'->',r'>', setlist) for setlist in setlists]
setlists_pretoken = [re.sub(r'([:,/[();>\n\t\r\xa0])',r' \1', setlist) for setlist in setlists_pretoken]
setlists_pretoken = [re.sub(r'([/("\n\t\r\xa0])',r'\1 ', setlist) for setlist in setlists_pretoken]
setlists_pretoken = [re.sub(r'  ', r' ', setlist) for setlist in setlists_pretoken]
setlists_pretoken = [x.lower() for x in setlists_pretoken]

In [9]:
example = setlists_pretoken[1871]
tokenizer = Tokenizer(lower=False,filters='$%+<=@\\^_`{|}~')
tokenizer.fit_on_texts(setlists_pretoken)
s = tokenizer.texts_to_sequences([example])[0]
t = tokenizer.sequences_to_texts([s])
print(s)
#print(len(tokenizer.index_word))
print(t)
print(' '.join(tokenizer.index_word[i] for i in s))

[1, 1, 1, 1, 1, 212, 495, 3, 409, 1, 19, 21, 11, 15, 1, 6, 6, 6, 6, 1760, 3880, 1207, 351, 11, 3881, 3, 266, 3, 23, 41, 681, 4097, 15, 1, 6, 6, 6, 1, 1, 1, 44, 13, 5, 163, 167, 12, 4, 323, 14, 3, 7, 77, 78, 22, 3, 16, 7, 65, 28, 3, 160, 161, 28, 3, 318, 1615, 39, 3, 7, 241, 57, 3, 225, 68, 115, 1, 1, 1, 26, 5, 2, 2, 2, 2, 12, 8, 843, 1320, 18, 263, 2, 2, 2, 2, 14, 8, 843, 1320, 18, 263, 35, 559, 544, 2, 2, 2, 2, 22, 8, 843, 1320, 18, 263, 35, 41, 59, 198, 8, 189, 2, 2, 2, 2, 28, 8, 1188, 1291, 18, 263, 2, 2, 2, 2, 39, 8, 1188, 1291, 18, 263, 17, 146, 17, 1369, 1830, 18, 203, 17, 146, 2, 2, 2, 2, 57, 8, 2766, 4098, 18, 263, 2, 2, 2, 2, 115, 8, 2766, 4098, 18, 263, 3, 17, 8, 1139, 1140, 1549, 137, 18, 6408, 1, 32, 34, 5, 1023, 32, 513, 200, 15, 1, 626, 20, 201, 15, 1, 40, 318, 1615, 6409, 9, 6410, 112, 10, 1, 29, 5, 2, 2, 2, 2, 30, 1, 1]
['\n \n \n \n \n july 11 , 2013 \n umphrey\'s mcgee — \r \n \t \t \t \t indian lookout country club — mariaville , ny , usa " camp bisco" \r \n \t \t \t

In [10]:
len(tokenizer.word_index)

7182

MAKING SEQUENCES

In [11]:
def make_sequences(setlists,
                   training_length=50,
                   lower=False,
                   filters='$%+<=@\\^_`{|}~'):
    #                      '!"$%*+-.;<=?@\\^_`{|}~'
    
    #create tokenizer object and train on all setlists
    tokenizer = Tokenizer(lower=lower,filters=filters)
    tokenizer.fit_on_texts(setlists)
    
    #create and store dictionaries between words and indices
    word_idx = tokenizer.word_index
    idx_word = tokenizer.index_word
    num_words = len(word_idx) + 1
    word_counts = tokenizer.word_counts
    
    print(f'There are {num_words} unique words.')
    
    #convert each setlist to sequence of indices
    sequences = tokenizer.texts_to_sequences(setlists)
    
    # Limit to sequences with more than training length tokens
    seq_lengths = [len(x) for x in sequences]
    over_idx = [
        i for i, l in enumerate(seq_lengths) if l > (training_length + 20)
    ]

    new_setlists = []
    new_sequences = []

    # Only keep sequences with more than training length tokens
    for i in over_idx:
        new_setlists.append(setlists[i])
        new_sequences.append(sequences[i])

    training_seq = []
    labels = []

    # Iterate through the sequences of tokens
    for seq in new_sequences:

        # Create multiple training examples from each sequence
        for i in range(training_length, len(seq)):
            # Extract the features and label
            extract = seq[i - training_length:i + 1]

            # Set the features and label
            training_seq.append(extract[:-1])
            labels.append(extract[-1])

    print(f'There are {len(training_seq)} training sequences.')

    # Return everything needed for setting up the model
    return word_idx, idx_word, num_words, word_counts, new_setlists, new_sequences, training_seq, labels

In [12]:
training_length = 50
word_idx, idx_word, num_words, word_counts, new_setlists, sequences, features, labels = make_sequences(
    setlists_pretoken, training_length=training_length, lower=True)

There are 7183 unique words.
There are 279284 training sequences.


In [13]:
num_words

7183

In [14]:
word_idx[''] = 0
idx_word[0] = ''

In [15]:
word_counts

OrderedDict([('\n', 44980),
             ('january', 157),
             ('21', 83),
             (',', 29853),
             ('1998', 61),
             ("umphrey's", 2660),
             ('mcgee', 2615),
             ('—', 5206),
             ('\r', 3615),
             ('\t', 11319),
             ('bridget', 3),
             ("mcguire's", 1),
             ('filling', 1),
             ('station', 4),
             ('south', 155),
             ('bend', 118),
             ('in', 2885),
             ('usa', 2509),
             ('set', 4732),
             ('1', 2654),
             (':', 12071),
             ('bob', 115),
             ('[1]', 4812),
             ('red', 261),
             ('baron', 71),
             ('[2]', 3718),
             ('>', 17093),
             ('divisions', 464),
             ('[3]', 2609),
             ('burning', 33),
             ('down', 226),
             ('the', 9997),
             ('house', 266),
             ('[4]', 1715),
             ('muffburger', 69),
    

In [16]:
len(features)

279284

In [17]:
features = np.array(features)

In [18]:
labels = np.array(labels)

In [19]:
features.shape

(279284, 50)

In [20]:
features = np.reshape(features, (features.shape[0], 50, 1))

In [21]:
features = features / float(num_words)

In [22]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

batch_size = 8
num_epochs = 20
# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_steps = len(features) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=lr_scheduler)

In [23]:
loss = tf.keras.losses.SparseCategoricalCrossentropy()


In [24]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(num_words, 256, input_length = 50),
#    tf.keras.layers.Conv1D(64, 5, padding='same', activation='relu'),
    tf.keras.layers.LSTM(512, return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.LSTM(256),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(num_words, activation='softmax')
])

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model.fit(features, labels, epochs=100, batch_size = 64, validation_split = 0.1)

Epoch 1/100
3928/3928 [==============================] - 446s 111ms/step - loss: 5.4326 - accuracy: 0.1266 - val_loss: 5.3512 - val_accuracy: 0.1358
Epoch 2/100
3928/3928 [==============================] - 275s 70ms/step - loss: 5.3536 - accuracy: 0.1269 - val_loss: 5.3775 - val_accuracy: 0.1358
Epoch 3/100
3928/3928 [==============================] - 200s 51ms/step - loss: 5.3418 - accuracy: 0.1269 - val_loss: 5.3944 - val_accuracy: 0.1358
Epoch 4/100
 180/3928 [>.............................] - ETA: 2:59 - loss: 5.3481 - accuracy: 0.1267

In [ ]:
history.history

In [ ]:
fig = plt.figure(figsize=(18,10))

plt.subplot(1,2,1)
plt.plot(range(len(history.history['loss'])),history.history['loss'])

plt.subplot(1,2,1)
plt.plot(range(len(history.history['val_loss'])),history.history['val_loss'])

plt.subplot(1,2,2)
plt.plot(range(len(history.history['accuracy'])),history.history['accuracy'])

plt.subplot(1,2,2)
plt.plot(range(len(history.history['val_accuracy'])),history.history['val_accuracy'])

plt.show()

In [ ]:
model.summary()

In [ ]:
lens = [len(x) for x in setlists_pretoken]
min(lens), np.argmin(lens)
setlists_pretoken[74]

In [ ]:
def next_words(model, 
              setlists_pretoken, #previous 50 words
              word_idx,
              idx_word,
            new_words=50):
    
    random_set_open= np.random.choice(setlists_pretoken)
    split_words = np.array(random_set_open.split(' '))
    
    while len(split_words) < 50:
        random_set_open= np.random.choice(setlists_pretoken)
        split_words = np.array(random_set_open.split(' '))
        
        
    #test case - remove after testing
    set_open = setlists_pretoken[1350]
    split_words = np.array(set_open.split(' '))
    #---------------------------------
    split_words = split_words[:50]
        
    
    
    result = [word for word in split_words] #list of new words, initialized with original 50 words
    
    for i in range(new_words):
        
        last_n_words = result[-new_words:]

        features = [[word_idx[word] for word in last_n_words]] #tokens
    
        features_reshaped = np.reshape(features,(1,50,1))
    
        prediction = model.predict(features_reshaped)
    
        label = np.random.choice(len(prediction[0]),p=prediction[0])
        
#        label = np.argmax(prediction[0]) #testing, edit this out later
    
        next_word = idx_word[label]
    
        result.append(next_word)
        
#        print('The last 50 words are {}'.format(last_n_words))
#        print('The result is {}'.format(result))
    
    return ' '.join(result)

In [ ]:
print(next_words(model, setlists_pretoken, word_idx, idx_word))